# RESUMEN EJECUTABLE - SESIÓN 3: Seguridad y Autenticación

**Contenido:** Cheat-sheet con snippets ejecutables y tablas de referencia rápida.

**Uso:** Este notebook es tu guía de consulta rápida para implementar autenticación en FastAPI.

## CONFIGURACIÓN INICIAL

In [ ]:
# Instalación
!pip install fastapi uvicorn passlib[bcrypt] PyJWT python-multipart python-dotenv -q

# Imports
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from passlib.context import CryptContext
import jwt
from jwt.exceptions import PyJWTError
from datetime import datetime, timedelta
from typing import Annotated
from pydantic import BaseModel
import os

print("✅ Listo para usar")

---

## 🔐 1. HASH DE PASSWORDS (bcrypt)

### Configuración

In [ ]:
# Crear contexto de hashing
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Funciones helper
def hash_password(password: str) -> str:
    """Hashea una contraseña"""
    return pwd_context.hash(password)

def verify_password(plain_password: str, hashed_password: str) -> bool:
    """Verifica si una contraseña coincide con el hash"""
    # ⚠️ bcrypt es intencionalmente LENTO (~300ms) para prevenir ataques de fuerza bruta
    return pwd_context.verify(plain_password, hashed_password)

### Ejemplo de uso

In [ ]:
# Hashear password
password_hash = hash_password("mi_password")
print(f"Hash: {password_hash}")

# Verificar password
es_valida = verify_password("mi_password", password_hash)
print(f"¿Password correcta? {es_valida}")

### 📊 Tabla: Algoritmos de hashing

| Algoritmo | Seguridad | Velocidad | Uso recomendado |
|-----------|-----------|-----------|------------------|
| **bcrypt** | ⭐⭐⭐⭐⭐ | Lenta (bueno) | ✅ Passwords |
| argon2 | ⭐⭐⭐⭐⭐ | Lenta (bueno) | ✅ Passwords |
| scrypt | ⭐⭐⭐⭐ | Lenta (bueno) | ✅ Passwords |
| MD5 | ⭐ | Rápida (malo) | ❌ NUNCA usar |
| SHA256 | ⭐⭐ | Rápida (malo) | ❌ NO para passwords |

---

## 🎫 2. JWT (JSON Web Tokens)

### Configuración

In [ ]:
# Configuración JWT (⚠️ En producción: usar os.getenv())
SECRET_KEY = "tu-clave-super-secreta-cambiar-en-produccion"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

### Generar token

In [ ]:
def crear_token(username: str) -> str:
    """Genera un JWT con el username y expiración"""
    payload = {
        "sub": username,  # "sub" (subject) es el claim estándar para identificador
        "exp": datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)

# Ejemplo
token = crear_token("usuario123")
print(f"Token: {token[:50]}...")

### Decodificar token

In [ ]:
def decodificar_token(token: str) -> dict:
    """Decodifica y valida un JWT"""
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except PyJWTError:
        raise ValueError("Token inválido o expirado")

# Ejemplo
payload = decodificar_token(token)
print(f"Username: {payload.get('sub')}")

### 📊 Estructura de un JWT

```
eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiOiJ1c2VyMTIzIiwiZXhwIjoxNjg5...abcdefg
└─────────── HEADER ──────────────┘ └───────── PAYLOAD ───────────┘ └─ SIGNATURE ─┘
```

| Parte | Contenido | Codificación |
|-------|-----------|-------------|
| **Header** | Algoritmo y tipo | Base64 |
| **Payload** | Claims (datos) | Base64 |
| **Signature** | Firma criptográfica | HMAC SHA256 |

### 📊 Claims estándar JWT

| Claim | Nombre | Descripción | Obligatorio |
|-------|--------|-------------|-------------|
| `sub` | Subject | Identificador del usuario | ✅ Sí |
| `exp` | Expiration | Timestamp de expiración | ✅ Sí |
| `iat` | Issued At | Cuándo se emitió | ❌ No |
| `iss` | Issuer | Quién emitió el token | ❌ No |
| `aud` | Audience | Para quién es el token | ❌ No |

---

## 🔓 3. OAUTH2 EN FASTAPI

### Modelos Pydantic

In [ ]:
class User(BaseModel):
    username: str
    email: str

class UserRegister(BaseModel):
    username: str
    email: str
    password: str

class Token(BaseModel):
    access_token: str
    token_type: str

### OAuth2PasswordBearer

In [ ]:
# Configura el esquema OAuth2
# tokenUrl="token" indica que POST /token es el endpoint de login
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Esta dependencia extrae el token del header: Authorization: Bearer <token>

### 📊 Diagrama: Flujo del token OAuth2

```
1. LOGIN (POST /token)
   Cliente → FastAPI: {username, password}
   FastAPI: Valida credenciales
   FastAPI → Cliente: {access_token: "eyJ...", token_type: "bearer"}

2. ACCESO PROTEGIDO (GET /usuarios/me)
   Cliente → FastAPI: Authorization: Bearer eyJ...
                        ↓
            [OAuth2PasswordBearer]  ← Extrae token del header
                        ↓
            [obtener_usuario_actual] ← Valida token + busca usuario
                        ↓
   FastAPI → Cliente: {username: "...", email: "..."}
```

### Dependencia de autenticación

In [ ]:
# Base de datos fake (en producción: usar BD real)
# ⚠️ Se borra al reiniciar el kernel del notebook (solo vive en RAM)
USERS_DB = {}

async def obtener_usuario_actual(token: str = Depends(oauth2_scheme)) -> User:
    """Valida el token y retorna el usuario"""
    try:
        # Decodificar token
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        
        # Buscar usuario en BD
        user_dict = USERS_DB.get(username)
        if user_dict is None:
            raise HTTPException(status_code=401, detail="Usuario no encontrado")
        
        return User(username=user_dict["username"], email=user_dict["email"])
    except PyJWTError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido",
            headers={"WWW-Authenticate": "Bearer"},
        )

# Tipo reutilizable
CurrentUser = Annotated[User, Depends(obtener_usuario_actual)]

---

## 🚀 4. ENDPOINTS COMPLETOS

### POST /register (Registro)

In [ ]:
app = FastAPI()

@app.post("/register", response_model=User)
async def registrar_usuario(user_data: UserRegister):
    # 1. Validar que no existe
    if user_data.username in USERS_DB:
        raise HTTPException(status_code=400, detail="Usuario ya existe")
    
    # 2. Hashear password
    hashed_password = hash_password(user_data.password)
    
    # 3. Guardar en BD
    USERS_DB[user_data.username] = {
        "username": user_data.username,
        "email": user_data.email,
        "hashed_password": hashed_password
    }
    
    # 4. Retornar usuario (sin password)
    return User(username=user_data.username, email=user_data.email)

### POST /token (Login)

In [ ]:
@app.post("/token", response_model=Token)
async def login(form_data: OAuth2PasswordRequestForm = Depends()):
    # 1. Buscar usuario
    user = USERS_DB.get(form_data.username)
    if not user:
        raise HTTPException(status_code=401, detail="Credenciales incorrectas")
    
    # 2. Verificar password
    if not verify_password(form_data.password, user["hashed_password"]):
        raise HTTPException(status_code=401, detail="Credenciales incorrectas")
    
    # 3. Generar token
    token = crear_token(user["username"])
    
    # 4. Retornar token
    return {"access_token": token, "token_type": "bearer"}

### GET /usuarios/me (Perfil protegido)

In [ ]:
@app.get("/usuarios/me", response_model=User)
async def obtener_mi_perfil(current_user: CurrentUser):
    """Endpoint protegido: requiere token válido"""
    return current_user

---

## 📊 5. TABLA COMPARATIVA: ESQUEMAS DE SEGURIDAD

| Método | Tipo | Dónde viaja | Stateful | Uso |
|--------|------|-------------|----------|-----|
| **Basic Auth** | Usuario:Password | Header (Base64) | No | APIs simples |
| **Bearer Token** | Token opaco | Header | Sí | APIs tradicionales |
| **JWT** | Token firmado | Header | No | ✅ APIs modernas |
| **OAuth2** | Token + Refresh | Header | Depende | Sistemas complejos |
| **API Key** | Clave fija | Header/Query | No | Servicios B2B |

---

## 📊 6. CÓDIGOS HTTP DE AUTENTICACIÓN

| Código | Nombre | Cuándo usar |
|--------|--------|-------------|
| **200** | OK | Login exitoso, perfil obtenido |
| **201** | Created | Usuario registrado |
| **400** | Bad Request | Usuario duplicado, datos inválidos |
| **401** | Unauthorized | ❌ Sin token, token inválido/expirado, credenciales incorrectas |
| **403** | Forbidden | Token válido pero sin permisos |
| **422** | Unprocessable | Validación Pydantic falló |

---

## 🧪 7. TESTING CON TESTCLIENT

### Pruebas básicas

In [ ]:
from fastapi.testclient import TestClient

client = TestClient(app)

# 1. Registro
response = client.post("/register", json={
    "username": "test",
    "email": "test@example.com",
    "password": "test123"
})
print(f"Registro: {response.status_code}")

# 2. Login
response = client.post("/token", data={
    "username": "test",
    "password": "test123"
})
token = response.json()["access_token"]
print(f"Login: {response.status_code}")

# 3. Acceso protegido
response = client.get("/usuarios/me", headers={
    "Authorization": f"Bearer {token}"
})
print(f"Perfil: {response.status_code}")
print(f"Usuario: {response.json()}")

---

## ⚠️ 8. ERRORES COMUNES Y SOLUCIONES

| Error | Causa | Solución |
|-------|-------|----------|
| `401 Unauthorized` | Token no enviado | Añadir header `Authorization: Bearer <token>` |
| `Signature has expired` | Token caducado | Generar nuevo token con POST /token |
| `Invalid token` | Token malformado o firma inválida | Verificar SECRET_KEY y ALGORITHM |
| `Usuario ya existe` | Intento de registro duplicado | Usar otro username |
| `422 Unprocessable` | JSON/form data mal formado | Verificar estructura de datos |
| Password no verifica | Hash incorrecto | Verificar que usas pwd_context.verify() |

---

## ✅ 9. CHECKLIST DE SEGURIDAD

### Desarrollo
- [ ] ✅ Passwords hasheadas con bcrypt
- [ ] ✅ Tokens JWT con expiración
- [ ] ✅ OAuth2PasswordBearer configurado
- [ ] ✅ Dependencia obtener_usuario_actual
- [ ] ✅ response_model para no exponer passwords

### Producción
- [ ] ⚠️ SECRET_KEY en variable de entorno
- [ ] ⚠️ HTTPS obligatorio (no HTTP)
- [ ] ⚠️ Rate limiting en /token
- [ ] ⚠️ Validación de email en registro
- [ ] ⚠️ Logs de intentos fallidos
- [ ] ⚠️ Refresh tokens implementados

---

## 🔗 10. SNIPPET: CONFIGURACIÓN CON .env

In [ ]:
# ⚠️ FORMA CORRECTA EN PRODUCCIÓN
import os
from dotenv import load_dotenv  # pip install python-dotenv

# Cargar variables del archivo .env
load_dotenv()

# Obtener SECRET_KEY desde variable de entorno
SECRET_KEY = os.getenv("SECRET_KEY")
if not SECRET_KEY:
    raise ValueError("SECRET_KEY no configurada en .env")

ALGORITHM = os.getenv("ALGORITHM", "HS256")  # Valor por defecto
ACCESS_TOKEN_EXPIRE_MINUTES = int(os.getenv("ACCESS_TOKEN_EXPIRE_MINUTES", "30"))

### Archivo .env (ejemplo)

```bash
# .env (NO SUBIR A GIT)
SECRET_KEY=tu-clave-generada-aleatoriamente-de-64-caracteres
ALGORITHM=HS256
ACCESS_TOKEN_EXPIRE_MINUTES=30
```

---

## 🎯 RESUMEN FINAL

### Flujo completo de autenticación

```
1. REGISTRO
   POST /register {username, email, password}
   → Hash password con bcrypt
   → Guardar en BD
   → Retornar 201 Created

2. LOGIN
   POST /token (form_data)
   → Buscar usuario
   → Verificar password con bcrypt
   → Generar JWT con PyJWT
   → Retornar {access_token, token_type}

3. ACCESO PROTEGIDO
   GET /usuarios/me (Authorization: Bearer <token>)
   → OAuth2PasswordBearer extrae token
   → obtener_usuario_actual valida token
   → Retornar datos del usuario
```

### Conceptos clave
- ✅ **Bcrypt:** Hash irreversible de passwords
- ✅ **JWT:** Token firmado con claims
- ✅ **OAuth2:** Estándar para autenticación en APIs
- ✅ **Depends:** Inyección de dependencias para validar tokens
- ✅ **401:** Código HTTP para errores de autenticación